In [ ]:
# install huggingface and datasets
!pip install -q datasets transformers
!pip install torch
!pip install pandas
!pip install pyarrow

In [2]:
from datasets import load_dataset, ClassLabel
from transformers import BertForTokenClassification, BertTokenizer, TrainingArguments, Trainer
import numpy as np
import torch
from sklearn import metrics


import pandas as pd
import datasets
from datasets.features import ClassLabel

In [3]:
torch.cuda.is_available()
device = torch.device("cuda")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
datapath = '/content/drive/MyDrive/Universitat/ML4NLP/v2.1/'

In [29]:
# There is no train split 
train_path = datapath + 'hipe2020/de/HIPE-2022-v2.1-hipe2020-train-de.tsv'
val_path = datapath + 'hipe2020/de/HIPE-2022-v2.1-hipe2020-dev-de.tsv'
test_path = datapath + 'hipe2020/de/HIPE-2022-v2.1-hipe2020-test_allmasked-de.tsv'

model = 'dbmdz/bert-base-historic-multilingual-cased'
shuffle = True

In [7]:
# import dataset from cloned git repo
def load_dataset(path):
    df = pd.read_csv(path, sep='\t', skip_blank_lines=False, engine='python', quoting=3)
    # error_bad_lines=False, 
    return df


tsv_train = load_dataset(train_path)
tsv_val = load_dataset(val_path)
tsv_test = load_dataset(test_path)

In [9]:
def simple_preprocess(dataframe):
    # Add end_of_document token in df
    dataframe = dataframe.dropna(subset=['TOKEN'])

    # Filter out metadata rows beginning with #
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('#')]
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('\t')]

    #transforming nan var from Float to string to use in (***)
    dataframe.MISC = dataframe.MISC.fillna('')

    return dataframe

In [10]:
tsv_train = simple_preprocess(tsv_train)
tsv_val = simple_preprocess(tsv_val)
tsv_test = simple_preprocess(tsv_test)

In [11]:
label_set = tsv_train['NE-COARSE-LIT'].unique()
# define the label mapping for NER
label_list = label_set.tolist()
label_list.append('_')
label_num = len(label_list)

labels = ClassLabel(num_classes=label_num, names=label_list)

In [12]:
def create_huggingface_file(dataframe):
    import pyarrow as pa
    import re
    from datasets import Dataset

    #creating dataset in json
    hug_out = []
    idx = 0 
    items = {'id': idx,'words':[ ], 'ner': [ ]}
    hug_out.append(items)
    for index, row in dataframe.iterrows():
        if  not re.search(r'EndOfSentence', row['MISC']):
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
        else:
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
            idx += 1
            items = {'id': idx,'words':[ ], 'ner': [ ]}
            hug_out.append(items)
    #filter hug_out out, delete items which has len(words) > 380
    #hug_out = filter(lambda x: len(x['words']) < 380, hug_out)
    #json to df
    hug_out = pd.DataFrame(hug_out)

    # delete all sentences that are too long
    #hug_out = hug_out[hug_out['words'].map(len) < 512] #why does not work? QA

    ### convert to Huggingface dataset
    hug_out = Dataset(pa.Table.from_pandas(hug_out))

    return hug_out

In [13]:
train = create_huggingface_file(tsv_train)
val = create_huggingface_file(tsv_val)
test = create_huggingface_file(tsv_test)

#look at training data
for i in range(10):
  print(train[i])

print(len(train))
print(len(val))
print(len(test))

{'id': 0, 'words': ['Frankreich', '.'], 'ner': [0, 1]}
{'id': 1, 'words': ['Gesetzgeber', '.'], 'ner': [1, 1]}
{'id': 2, 'words': ['Den', '19', '.', 'Niv', '.'], 'ner': [1, 1, 1, 1, 1]}
{'id': 3, 'words': ['(', '8', '.', 'Jän', '.', ')', 'ward', 'die', 'Staatskleidung', 'der', 'Sekretär', '-', 'Redacteurs', ',', 'der', 'Staatsbothen', 'und', 'Thorwächter', 'für', 'beyde', 'Räthe', 'bestimmt', '.'], 'ner': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'id': 4, 'words': ['Auf', 'Talot', "'", 's', 'Vorschlag', 'beschloß', 'man', 'über', 'den', 'constitutionellen', 'Umkreis', ',', 'welchen', 'das', 'gesetzgebende', 'Corps', 'in', 'Zukunft', 'inne', 'haben', 'soll', ',', 'fol', '¬', 'gendes', ':', 'Vom', 'Tage', 'an', ',', 'da', 'der', 'Rath', 'der', '500', 'in', 'sei', '¬', 'nen', 'neuen', 'Pallast', 'installirt', 'seyn', 'wird', ',', 'sind', 'die', 'äußerlichen', 'Bezirke', 'für', 'beyde', 'Räthe', 'folgendermas', '¬', 'sen', 'firir', ':', 'Rarh', 'der', 'Alten',

In [14]:
tokenizer = BertTokenizer.from_pretrained(model)

Downloading:   0%|          | 0.00/207k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/561 [00:00<?, ?B/s]

In [15]:
train = train.filter(lambda example, idx: idx != len(train)-1, with_indices=True)
val = val.filter(lambda example, idx: idx != len(val)-1, with_indices=True)
test = test.filter(lambda example, idx: idx != len(test)-1, with_indices=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [16]:
def tokenize_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]
    tokenized_ner = []

    for sent_words, sent_nes in zip(tokenized_words, batch["ner"]):
        tokenized_ner.append(
            [labels.str2int("O")]  # BOS symbol
            + [
                ne
                for subwords, ne in zip(sent_words, sent_nes)
                for _ in range(len(subwords))
            ]
            + [labels.str2int("O")]  # EOS symbol
        )

    # Padding with "O"
    tokenized["labels"] = [(ner + [labels.str2int("O")] * (100 - len(ner)))[:100] for ner in tokenized_ner]
    tokenized["subwords"] = tokenized_words
    return tokenized

def tokenize_nolabel_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]

    tokenized["subwords"] = tokenized_words
    return tokenized

#tiny_train_tokenized = tiny_train.map(tokenize_func, batched=True, batch_size=50)
#tiny_test_tokenized = tiny_test.map(tokenize_nolabel_func, batched=True, batch_size=50)

train_tokenized = train.map(tokenize_func, batched=True, batch_size=50)
val_tokenized = val.map(tokenize_func, batched=True, batch_size=50)
test_tokenized = test.map(tokenize_nolabel_func, batched=True, batch_size=50)


  0%|          | 0/70 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

In [ ]:
training_args = TrainingArguments(
    output_dir = "/content/drive/MyDrive/Universitat/ML4NLP/bert_europeana_en"
)

In [30]:
if shuffle:
  train_tokenized.shuffle()

In [31]:
model = BertForTokenClassification.from_pretrained(model, num_labels=labels.num_classes)
# trainer = Trainer(model=model, args=training_args, train_dataset=train_tokenized)
trainer = Trainer(model=model, train_dataset=train_tokenized)
trainer.train()

loading configuration file https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/5ce7e041308a3a4d2a07e1655ad227e92ce92724a53cbba35dabc5870332286f.52c144249cc94a2c4412d3bb497665d3d77c1389cfc2bdce33b2e6a8d3220542
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5

Step,Training Loss
500,0.082900
1000,0.023200


Saving model checkpoint to tmp_trainer/checkpoint-500
Configuration saved in tmp_trainer/checkpoint-500/config.json
Model weights saved in tmp_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to tmp_trainer/checkpoint-1000
Configuration saved in tmp_trainer/checkpoint-1000/config.json
Model weights saved in tmp_trainer/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1302, training_loss=0.04319867047662925, metrics={'train_runtime': 244.0737, 'train_samples_per_second': 42.651, 'train_steps_per_second': 5.334, 'total_flos': 531317414088000.0, 'train_loss': 0.04319867047662925, 'epoch': 3.0})

In [32]:
predictions = trainer.predict(test_tokenized)

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subwords, words, ner, id. If subwords, words, ner, id are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1215
  Batch size = 8


In [33]:
outfilepath = f"/content/drive/MyDrive/e_ML4NLP2/Outputs/hipe2020_de/europeana_multiling_bert_cased{'_shuffled' if shuffle else ''}.tsv"
outfilepath

'/content/drive/MyDrive/e_ML4NLP2/Outputs/hipe2020_de/europeana_multiling_bert_cased_shuffled.tsv'

In [34]:
outfile = open(outfilepath, "w")
outfile.write("TOKEN\tNE-COARSE-LIT\tNE-COARSE-METO\tNE-FINE-LIT\tNE-FINE-METO\tNE-FINE-COMP\tNE-NESTED\tNEL-LIT\tNEL-METO\tMISC")

for pred, tinput in zip(predictions.predictions, test_tokenized):
    pred_word_labels = []

    try:
        sent_len = tinput["input_ids"].index(0)
    except ValueError:
        sent_len = len(tinput["input_ids"])
    # Start at 1 to skip pred for cls token
    i = 1

    for word in tinput['subwords']:
        
        word_score = np.zeros_like(pred[0])
        eos_reached = False
        try:
            for subword in word:
                word_score += pred[i]
                i += 1
            label = labels.int2str(int(np.argmax(word_score)))
            pred_word_labels.append(label)
        except IndexError:
            pred_word_labels.append("O")

    for token, label in zip(tinput['words'], pred_word_labels):
        outfile.write("\n" + f"{token}\t{label}\t" + ('\t'.join(['_'] * 8)))

outfile.close()

In [35]:
accuracy = datasets.load_metric("accuracy")
f1_metric = datasets.load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    refs = labels.flatten()
    predictions = np.argmax(logits, axis=-1)
    print(logits.shape)
    print(predictions.shape)
    pred = predictions.flatten()
    return {
        "accuracy": accuracy.compute(predictions=pred, references=refs)["accuracy"],
        "f1_micro": f1_metric.compute(predictions=pred, references=refs, average="micro")["f1"],
        "f1_macro": f1_metric.compute(predictions=pred, references=refs, average="macro")["f1"],
    }

In [36]:
Trainer(
    model=model,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
).evaluate()

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subwords, words, ner, id. If subwords, words, ner, id are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1202
  Batch size = 8


(1202, 100, 12)
(1202, 100)


{'eval_accuracy': 0.9891098169717139,
 'eval_f1_macro': 0.7322631667326764,
 'eval_f1_micro': 0.9891098169717139,
 'eval_loss': 0.044546693563461304,
 'eval_runtime': 9.7931,
 'eval_samples_per_second': 122.74,
 'eval_steps_per_second': 15.419}